In [ ]:

#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import time, re, json, csv, sys
from dataclasses import dataclass
from typing import Iterator, Optional, Dict, Any, List
from urllib.parse import urljoin

import requests
from bs4 import BeautifulSoup

In [ ]:
@dataclass
class Config:
    page_url_pattern: str
    item_selector: str
    title_selector: Optional[str]
    price_selector: Optional[str]
    url_selector: Optional[str]
    next_selector: Optional[str] = None

    def parse_price(self, text: Optional[str]) -> Optional[float]:
        if not text:
            return None
        m = re.search(r"([0-9]+(?:[.,][0-9]+)?)", text)
        return float(m.group(1).replace(",", ".")) if m else None


In [ ]:
QUOTES_CFG = Config(
    page_url_pattern="https://quotes.toscrape.com/page/{page}/",
    item_selector=".quote",
    title_selector=".text",
    price_selector=None,
    url_selector="span a",
    next_selector=".next a"
)

# DLA BOOKS: używamy "next", więc ważny jest next_selector i start_url = index.html
BOOKS_CFG = Config(
    page_url_pattern="",  # w trybie next nieużywane
    item_selector="article.product_pod",
    title_selector="h3 a",
    price_selector=".price_color",
    url_selector="h3 a",
    next_selector="li.next a"
)

In [ ]:
# -------------------- HTTP --------------------

HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; PaginatorScraper/1.1)"}

def get(url: str, retries: int = 3, backoff: float = 0.5) -> requests.Response:
    last_exc = None
    for i in range(retries):
        try:
            r = requests.get(url, headers=HEADERS, timeout=15)
            r.raise_for_status()
            return r
        except requests.RequestException as e:
            last_exc = e
            if i < retries - 1:
                time.sleep(backoff * (i + 1))
    raise last_exc  # type: ignore

def text_or_none(node) -> Optional[str]:
    return node.get_text(" ", strip=True) if node else None

def abs_href(root: BeautifulSoup, el) -> Optional[str]:
    if not el:
        return None
    href = el.get("href")
    if not href:
        return None
    base_tag = root.select_one("base[href]")
    base_url = base_tag.get("href") if base_tag else ""
    return urljoin(base_url, href)

In [ ]:
# -------------------- ITERACJA --------------------

def iterate_by_pattern(cfg: Config, start_page: int, max_pages: int) -> Iterator[str]:
    for p in range(start_page, start_page + max_pages):
        yield cfg.page_url_pattern.format(page=p)

def iterate_by_next(start_url: str, cfg: Config, max_pages: int) -> Iterator[str]:
    url = start_url
    pages = 0
    while url and pages < max_pages:
        yield url
        pages += 1
        soup = BeautifulSoup(get(url).text, "html.parser")
        nxt = soup.select_one(cfg.next_selector) if cfg.next_selector else None
        url = urljoin(url, nxt.get("href")) if (nxt and nxt.get("href")) else None


In [ ]:
# -------------------- EKSTRAKCJA --------------------

def extract_items(page_html: str, cfg: Config) -> List[Dict[str, Any]]:
    soup = BeautifulSoup(page_html, "htmlparser") if False else BeautifulSoup(page_html, "html.parser")
    out: List[Dict[str, Any]] = []
    for item in soup.select(cfg.item_selector):
        # tytuł
        title = None
        if cfg.title_selector:
            tnode = item.select_one(cfg.title_selector)
            title = tnode.get("title") if (tnode and tnode.get("title")) else text_or_none(tnode)
        # cena
        price_val = None; price_raw = None
        if cfg.price_selector:
            pnode = item.select_one(cfg.price_selector)
            price_raw = text_or_none(pnode)
            price_val = cfg.parse_price(price_raw)
        # link
        link_abs = None
        if cfg.url_selector:
            anode = item.select_one(cfg.url_selector)
            link_abs = abs_href(soup, anode)
        out.append({"title": title, "price": price_val, "price_raw": price_raw, "url": link_abs})
    return out


In [ ]:
# -------------------- ZAPIS --------------------

def save_csv(rows: List[Dict[str, Any]], path: str) -> None:
    if not rows:
        print(f"Brak danych → {path} nie zapisany.")
        return
    keys = set().union(*(r.keys() for r in rows))
    cols = sorted(keys)
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=cols)
        w.writeheader()
        for r in rows:
            w.writerow(r)
    print(f"CSV zapisany: {path} ({len(rows)} rekordów)")

def save_jsonl(rows: List[Dict[str, Any]], path: str) -> None:
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")
    print(f"JSONL zapisany: {path} ({len(rows)} rekordów)")


In [ ]:
# -------------------- RUN --------------------

def run(mode: str, cfg: Config, start_url: Optional[str] = None,
        start_page: int = 1, max_pages: int = 50, delay: float = 0.3) -> List[Dict[str, Any]]:
    results: List[Dict[str, Any]] = []

    if mode == "pattern":
        urls = list(iterate_by_pattern(cfg, start_page, max_pages))
    elif mode == "next":
        if not start_url:
            raise ValueError("Dla mode='next' wymagany jest start_url.")
        urls = list(iterate_by_next(start_url, cfg, max_pages))
    else:
        raise ValueError("mode ∈ {'pattern','next'}")

    print(f"Do odwiedzenia {len(urls)} stron...")
    for i, url in enumerate(urls, 1):
        try:
            r = get(url)
            page_rows = extract_items(r.text, cfg)
            if not page_rows and mode == "pattern":
                print(f"[{i}] {url} – pusto, przerywam.")
                break
            results.extend(page_rows)
            print(f"[{i}] {url} → {len(page_rows)} wyników (suma: {len(results)})")
            time.sleep(delay)
        except Exception as e:
            print(f"[{i}] Błąd przy {url}: {e}", file=sys.stderr)
    return results


In [ ]:
if __name__ == "__main__":
    # DEMO 1: quotes (działa jak wcześniej)
    quotes = run(
        mode="next",
        cfg=QUOTES_CFG,
        start_url="https://quotes.toscrape.com/page/1/",
        max_pages=50,
        delay=0.2
    )
    save_csv(quotes, "quotes_results.csv")
    save_jsonl(quotes, "quotes_results.jsonl")

    # DEMO 2: books – KLUCZOWA ZMIANA → tryb 'next' + start od index.html
    books = run(
        mode="next",
        cfg=BOOKS_CFG,
        start_url="https://books.toscrape.com/catalogue/category/books/travel_2/index.html",
        max_pages=50,
        delay=0.2
    )
    save_csv(books, "books_results.csv")
    save_jsonl(books, "books_results.jsonl")

    print("Gotowe.")

Do odwiedzenia 10 stron...
[1] https://quotes.toscrape.com/page/1/ → 10 wyników (suma: 10)
[2] https://quotes.toscrape.com/page/2/ → 10 wyników (suma: 20)
[3] https://quotes.toscrape.com/page/3/ → 10 wyników (suma: 30)
[4] https://quotes.toscrape.com/page/4/ → 10 wyników (suma: 40)
[5] https://quotes.toscrape.com/page/5/ → 10 wyników (suma: 50)
[6] https://quotes.toscrape.com/page/6/ → 10 wyników (suma: 60)
[7] https://quotes.toscrape.com/page/7/ → 10 wyników (suma: 70)
[8] https://quotes.toscrape.com/page/8/ → 10 wyników (suma: 80)
[9] https://quotes.toscrape.com/page/9/ → 10 wyników (suma: 90)
[10] https://quotes.toscrape.com/page/10/ → 10 wyników (suma: 100)
CSV zapisany: quotes_results.csv (100 rekordów)
JSONL zapisany: quotes_results.jsonl (100 rekordów)
Do odwiedzenia 1 stron...
[1] https://books.toscrape.com/catalogue/category/books/travel_2/index.html → 11 wyników (suma: 11)
CSV zapisany: books_results.csv (11 rekordów)
JSONL zapisany: books_results.jsonl (11 rekordów)
Gotowe.
